In [33]:
import pandas as pd
import numpy as np
import plotly.express as px
import klib as kl

# impute missing values with iterative imputer 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [34]:
products_data = pd.read_csv('data/regression_clean_data.csv')

data = pd.read_csv('data/site_changes_cleaned.csv')



# create a date column from month and year and add day as 1
data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))
data = kl.clean_column_names(data)

data.head()

,month,year,incentive,apr,notes,bank,notes_1,tag,site_changes,date
0,1,2021,0,0.0,NaN,abanca,NaN,NaN,NaN,2021-01-01
1,6,2021,150,0.0,800-1200 deposit,abanca,NaN,increase incentive,NaN,2021-06-01
2,8,2021,150,0.0,800-1200 deposit,abanca,NaN,incentive decrease,NaN,2021-08-01
3,4,2022,300,0.0,if over 1200 deposit,abanca,NaN,"incentive increase, apr decrease",NaN,2022-04-01
4,12,2022,300,0.0,mejor cambia de el sitio,abanca,NaN,NaN,major site change,2022-12-01


In [35]:
# rename tags

data.tag = data.tag.replace({'apr increase':'increase apr', 'incentive increase':'increase incentive', 'decrease incentive':'incentive decrease',  'decrease incentive ':'incentive decrease',
'incentive decrease, apr increase':'decrease incentive, increase apr',
'incentive increase, apr decrease':'increase incentive, decrease apr',
'incentive increase, apr increase':'increase incentive, increase apr',
'incentive decrease, apr decrease':'decrease incentive, decrease apr'})

       
data.tag.value_counts()

tag
increase incentive                  15
increase apr                        15
incentive decrease                   3
decrease apr                         3
decrease incentive, increase apr     2
increase incentive, decrease apr     1
increase incentive, increase apr     1
decrease incentive, decrease apr     1
temporary incentive                  1
Name: count, dtype: int64

In [36]:
mapping_dict = {
    'abanca': 'abanca',
    'banc sabadell': 'banc sabadell',
    'bancosabadell': 'banc sabadell',
    'bancosantander': 'santander',
    'bancsabadell': 'banc sabadell',
    'sabadell': 'banc sabadell',
    'bank': 'bank',
    'bankinter': 'bankinter',
    'bbva': 'bbva',
    'caixabank': 'caixabank',
    'caixa_bank': 'caixabank',
    'date': 'date',
    'evobanco': 'evobanco',
    'evo_banco': 'evobanco',
    'imagin': 'imagin',
    'ing': 'ing',
    'ing-bank': 'ing',
    'ing_bank': 'ing',
    'kutxabank': 'kutxabank',
    'liberbank': 'liberbank',
    'myinvestor': 'myinvestor',
    'n26': 'n26',
    'n26.com': 'n26',
    None: None,  # handling nan (not a number)
    'openbank': 'openbank',
    'orangebank': 'orangebank',
    'pibank': 'pibank',
    'renaultbank': 'renaultbank',
    'revolut': 'revolut',
    'santander': 'santander',
    'value': 'value'
}

# map santander to bancosantander in data 
data.bank = data.bank.map(mapping_dict)
products_data.bank = products_data.bank.map(mapping_dict)



In [37]:
# cast date to datetime
products_data['date'] = pd.to_datetime(products_data['date'])
products_data = products_data[['date', 'bank', 'web_traffic']]

data['date'] = pd.to_datetime(data['date']) 

In [38]:

# attach product_data to data
data = pd.merge(products_data, data, on=['bank', 'date'], how='left')

# drop the 124 row 
data = data.drop(124)

data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))


# for some reason banks are not having the dates attaching properly to the bank data so we have to re add all of the dates to the banks

for bank in data.bank.unique():
    data.loc[data.bank == bank, 'date'] = list(products_data.date.unique())


In [39]:

data.tag = data.tag.fillna('')
apr_increase_dates = data[data.tag.str.contains('increase apr')]
incentive_increase_dates = data[data.tag.str.contains('increase incentive')]


all_dates = data.date.unique()


In [40]:
temp_products_data = products_data.copy()

In [41]:
#apr_increase_dates = temp_products_data.merge(apr_increase_dates, on=['date','bank'], how='left')
#incentive_increase_dates = temp_products_data.merge(incentive_increase_dates, on=['date','bank'], how='left')

apr_increase_dates = apr_increase_dates.ffill().bfill()
incentive_increase_dates = incentive_increase_dates.ffill().bfill()

In [42]:
# plot the apr increases for all banks where apr sum is not 0 in plotly express 
fig = px.line(apr_increase_dates, x='date', y='apr', color='bank', title='APR Increases')
fig.show()